In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
from joblib import Parallel, delayed
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET,) #blockchain="coinbase_base")
C.DEFAULT_MIN_PROFIT_GAS_TOKEN = 0.005

setup_bot = CarbonBot(ConfigObj=C)
pools = None

### 
"""
Put path to log file here >>>
"""
###

path = os.path.normpath("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json")
print(f"path={path}")
with open(path) as f:
    pools = json.load(f)



# with open("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json") as f:
#     pools = json.load(f)

flashloan_tokens = ["BNT-FF1C","WETH-6Cc2","USDC-eB48","LINK-86CA"]
#flashloan_tokens = ["WETH-0006,USDC-2913"]

pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
uniswap_v3_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v3_event_mappings.csv", low_memory=False)

tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,pancakeswap_v2,pancakeswap_v3,uniswap_v2,uniswap_v3"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    #uniswap_v3_event_mappings=uniswap_v3_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
#bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
#flashloan_tokens = bot.setup_flashloan_tokens(None)
#flashloan_tokens = ['WBTC-2c599', 'USDC-eB48', 'LINK-86CA', 'USDT-1ec7']


CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()


2023-11-19 10:47:35,938 [fastlane:INFO] - 
2023-11-19 10:47:35,939 [fastlane:INFO] - **********************************************
2023-11-19 10:47:35,939 [fastlane:INFO] - The logging path is set to: logs/20231119-104735\bot.log
2023-11-19 10:47:35,940 [fastlane:INFO] - **********************************************
2023-11-19 10:47:35,940 [fastlane:INFO] - 


path=fastlane_bot\data\tests\latest_pool_data_testing_balancer.json


2023-11-19 10:47:38,225 [fastlane:INFO] - Retrieved 208 carbon pairs from contract
2023-11-19 10:47:40,259 [fastlane:INFO] - Time taken to add initial pools: 0.09514498710632324
2023-11-19 10:47:40,262 [fastlane:INFO] - Initializing the bot...
2023-11-19 10:47:40,462 [fastlane:INFO] - uniswap_v3: 470
2023-11-19 10:47:40,463 [fastlane:INFO] - sushiswap_v2: 63
2023-11-19 10:47:40,463 [fastlane:INFO] - uniswap_v2: 708
2023-11-19 10:47:40,464 [fastlane:INFO] - bancor_v2: 0
2023-11-19 10:47:40,464 [fastlane:INFO] - bancor_v3: 27
2023-11-19 10:47:40,465 [fastlane:INFO] - bancor_pol: 51
2023-11-19 10:47:40,465 [fastlane:INFO] - carbon_v1: 185
2023-11-19 10:47:40,466 [fastlane:INFO] - pancakeswap_v2: 0
2023-11-19 10:47:40,467 [fastlane:INFO] - pancakeswap_v3: 0
2023-11-19 10:47:40,467 [fastlane:INFO] - balancer: 63
2023-11-19 10:47:40,599 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 1024
2023-11-19 10:47:40,601 [fastlane:INFO] - sushiswap_v2_zero_liquidity_pools: 39
2023-11-19 10:47:40,6

# Balancer [NB056]

## Test_get_arb_finder

## Test_Expected_Output

In [3]:
# ["single", "multi", "triangle", "multi_triangle", "bancor_v3", "b3_two_hop"]
# multi_pairwise_bal
# multi_pairwise_pol
# multi_triangle_pol
arb_mode = "multi"

arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()


## Test_Single_Direction_Carbon_Curves

In [4]:
print(len(r))

3


In [5]:
r = [arb for arb in r if len(arb[2]) >= 2]

In [6]:
print(len(r))

3


In [7]:
r.sort(key=lambda x: x[0], reverse=True)

In [8]:
import random
import time
from _decimal import Decimal
from dataclasses import dataclass, asdict, field
from datetime import datetime
from typing import List, Dict, Tuple, Any, Callable
from typing import Optional

from fastlane_bot.config import Config
from fastlane_bot.helpers import (
    TxSubmitHandler,
    TxSubmitHandlerBase,
    TxReceiptHandler,
    TxReceiptHandlerBase,
    TxRouteHandler,
    TxRouteHandlerBase,
    TxHelpers,
    TxHelpersBase,
    TradeInstruction,
    Univ3Calculator,
    RouteStruct,
)
from fastlane_bot.helpers.routehandler import maximize_last_trade_per_tkn #,BalancerInputTooLargeError
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.tools.optimizer import CPCArbOptimizer
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.modes.pairwise_multi import FindArbitrageMultiPairwise
from fastlane_bot.modes.pairwise_single import FindArbitrageSinglePairwise
from fastlane_bot.modes.triangle_multi import ArbitrageFinderTriangleMulti
from fastlane_bot.modes.triangle_single import ArbitrageFinderTriangleSingle
from fastlane_bot.modes.triangle_single_bancor3 import ArbitrageFinderTriangleSingleBancor3
from fastlane_bot.modes.triangle_bancor_v3_two_hop import ArbitrageFinderTriangleBancor3TwoHop
from fastlane_bot.utils import num_format, log_format, num_format_float


exchange_specific_arbs = []

for arb in r:

    (
        best_profit,
        best_trade_instructions_df,
        best_trade_instructions_dic,
        best_src_token,
        best_trade_instructions,
    ) = arb

    # Order the trade instructions
    (
        ordered_trade_instructions_dct,
        tx_in_count,
    ) = bot._simple_ordering_by_src_token(
        best_trade_instructions_dic, best_src_token
    )

    # Scale the trade instructions
    ordered_scaled_dcts = bot._basic_scaling(
        ordered_trade_instructions_dct, best_src_token
    )

    # Convert the trade instructions
    ordered_trade_instructions_objects = bot._convert_trade_instructions(
        ordered_scaled_dcts
    )

    # Create the tx route handler
    tx_route_handler = bot.TxRouteHandlerClass(
        trade_instructions=ordered_trade_instructions_objects
    )

    # Aggregate the carbon trades
    agg_trade_instructions = (
        tx_route_handler.aggregate_carbon_trades(ordered_trade_instructions_objects)
        if bot._carbon_in_trade_route(ordered_trade_instructions_objects)
        else ordered_trade_instructions_objects
    )

    # Calculate the trade instructions
    #try:
    calculated_trade_instructions = tx_route_handler.calculate_trade_outputs(agg_trade_instructions)
    #    )
    #except Exception:
    #except BalancerInputTooLargeError:
        #continue

    #exchange_specific_arbs.append(calculated_trade_instructions)
    # for trade in calculated_trade_instructions:
    #     if "balancer" not in trade.exchange_name:
    #         exchange_specific_arbs.append(calculated_trade_instructions)
    exchange_specific_arbs.append(calculated_trade_instructions)


2023-11-19 10:47:42,364 [fastlane:INFO] - 


2023-11-19 10:47:42,364 [fastlane:INFO] - [calculate_trade_outputs REMOVE] starting Carbon trade calculations, 4 trades, remaining_tkn_in = 1.6040228239306238
2023-11-19 10:47:42,397 [fastlane:INFO] - [calculate_trade_outputs REMOVE] calculated trade, 1.089554130737549800 WETH-6Cc2 into trade, remaining=0.514468693193074000
2023-11-19 10:47:42,430 [fastlane:INFO] - [calculate_trade_outputs REMOVE] calculated trade, 0.321695218445733200 WETH-6Cc2 into trade, remaining=0.192773474747340800
2023-11-19 10:47:42,464 [fastlane:INFO] - [calculate_trade_outputs REMOVE] calculated trade, 0.105540831068996340 WETH-6Cc2 into trade, remaining=0.087232643678344460
2023-11-19 10:47:42,551 [fastlane:INFO] - [calculate_trade_outputs REMOVE] calculated trade, 0.087232643678344460 WETH-6Cc2 into trade, remaining=0E-18
2023-11-19 10:47:42,809 [fastlane:INFO] - 


2023-11-19 10:47:42,809 [fastlane:INFO] - [calculate_trade_outputs REMOVE] starting Carbon trade c

In [9]:
#exchange_specific_arbs.sort(key=lambda x: (x[-1].amtout - x[0].amtin), reverse=True)

In [10]:
print("Number of arbs: ",len(exchange_specific_arbs))
for arb in exchange_specific_arbs:

    for trade in arb:
        print(f"trade {trade.amtin} {trade.tknin} for {trade.amtout} {trade.tknout} on {trade.exchange_name}")
    profit = arb[-1].amtout - arb[0].amtin
    print(f"profit = {profit} {arb[-1].tknout}\n\n")

Number of arbs:  3
trade 1.604022823930623800 WETH-6Cc2 for 2538.734701 USDC-eB48 on carbon_v1
trade 2538.734701 USDC-eB48 for 151.821496643532223607 WETH-6Cc2 on uniswap_v2
profit = 150.217473819601599807 WETH-6Cc2


trade 26.736689 USDC-eB48 for 1.599050631768161897 WETH-6Cc2 on uniswap_v2
trade 1.599050631768161897 WETH-6Cc2 for 2530.971707 USDC-eB48 on carbon_v1
profit = 2504.235018 USDC-eB48


trade 45029.170793 USDC-eB48 for 45021.261344 USDT-1ec7 on carbon_v1
trade 45021.261344 USDT-1ec7 for 45017.413216 USDC-eB48 on uniswap_v3
profit = -11.757577 USDC-eB48


